# **Demo:** Net2WiderNet on ImageNet with Inception-V2

The following demo shows how to apply Net2WiderNet to Inception-V2 in order to increase the number of output filters in each layer of the Inception blocks. The input image shape is the one of CIFAR-10 but the network and the Net2WiderNet algorithm can be applied to any other image size.

In [1]:
# Import libraries
import torch
import numpy as np
import torchinfo

# Import custom modules and packages
from inceptionv2 import GoogleNetBN
import params.inceptionv2_cifar
import net2net.net2net_wider

### 1. Create an Inception-V2 model narrower than the original one

We start by creating an Inception-V2 model, narrower than the standard model: the number of convolution channels at each layer within all Inception modules is reduced by a factor of $\sqrt{0.3}$. The rest of the network remains the same.

In [2]:
# Create a downsized version of the Inception-V2 network
# (with 10 classes instead of 1000 for demo purposes)
model = GoogleNetBN(nb_classes=10, inception_factor=np.sqrt(0.3))

# Create a random input
x = torch.randn(1,
                params.inceptionv2_cifar.NB_CHANNELS,
                *params.inceptionv2_cifar.IMAGE_SHAPE)

# Compute the output of the teacher network
# (forward pass to initialize the Lazy modules)
y_teacher = model(x)

/home/tom/miniconda3/envs/mla/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


### 2. Recover the standard architecture of Inception-V2 using the Net2WiderNet algorithm

We then apply the Net2WiderNet algorithm to the narrower model in order to recover the standard architecture of Inception-V2. The algorithm is applied to the Inception modules and the fully-connected layer only, since the rest of the network is already standard. The weights and biases of the student model (the wider one) are initialized with those of the teacher model (the narrower one), in such a way that the output of the student model is the same as the output of the teacher model for the same input at initialization.

In [3]:
# Instantiate a Net2Net object from a (pre-trained) model
net2net = net2net.net2net_wider.Net2Net(teacher_network=model)

# Get the list of widening operations
wider_operations = params.inceptionv2_cifar.wider_operations

# Add some noise to the copied weights (optional)
sigma = 0.  # Standard deviation of the noise


# Go through the list of widening operations
for key in wider_operations.keys():

    print("Widening operation: ", key)
    
    # Get the parameters of the wider operation
    target_conv_layers = wider_operations[key]["target_conv_layers"]
    next_layer = wider_operations[key]["next_layers"]
    new_width = wider_operations[key]["width"]
    batch_norm_layers = wider_operations[key]["batch_norm_layers"]

    # Widen a layer of the network
    net2net.net2wider(target_conv_layers=target_conv_layers,
                      next_layers=next_layer,
                      width=new_width,
                      batch_norm_layers=batch_norm_layers,
                      sigma=sigma)


# Compute the output of the student network
y_student = net2net.student_network(x)

Widening operation:  inception1.1
Widening operation:  inception1.2
Widening operation:  inception1.3
Widening operation:  inception2.1
Widening operation:  inception2.2
Widening operation:  inception2.3
Widening operation:  inception3.1
Widening operation:  inception3.2
Widening operation:  inception3.3
Widening operation:  inception4.1
Widening operation:  inception4.2
Widening operation:  inception4.3
Widening operation:  inception5.1
Widening operation:  inception5.2
Widening operation:  inception5.3
Widening operation:  inception6.1
Widening operation:  inception6.2
Widening operation:  inception6.3
Widening operation:  inception7.1
Widening operation:  inception7.2
Widening operation:  inception7.3
Widening operation:  inception8.1
Widening operation:  inception8.2
Widening operation:  inception8.3
Widening operation:  inception9.1
Widening operation:  inception9.2
Widening operation:  inception9.3


### 3. Check that the student and teacher models have the same output for the same input

We check that the output of the student model is the same as the output of the teacher model for the same input at initialization. They can be slightly different if some noise has been added to the weights of the student model during the initialization.

In [4]:
# The outputs should be the same
print("Teacher output: ", y_teacher)
print("Student output: ", y_student, "\n")

Teacher output:  tensor([[-0.1073,  0.0501,  0.1479,  0.3136, -0.1870, -0.3176,  0.3336,  0.2729,
         -0.0309,  0.3061]], grad_fn=<AddmmBackward0>)
Student output:  tensor([[-0.1073,  0.0501,  0.1479,  0.3136, -0.1870, -0.3176,  0.3336,  0.2729,
         -0.0309,  0.3061]], grad_fn=<AddmmBackward0>) 



### 4. Have a look at the student and teacher architectures

We start by displaying the architecture of the teacher model. We can check that the number of convolution channels at each layer within all Inception modules is reduced by a factor of $\sqrt{0.3}$. The model has $1.886.577$ trainable parameters.

In [6]:
# Display the architecture of the student network
torchinfo.summary(model, input_size=(1,
                                     params.inceptionv2_cifar.NB_CHANNELS,
                                     *params.inceptionv2_cifar.IMAGE_SHAPE))

Layer (type:depth-idx)                        Output Shape              Param #
GoogleNetBN                                   [1, 10]                   --
├─Sequential: 1-1                             [1, 10]                   --
│    └─Sequential: 2-1                        [1, 64, 9, 9]             --
│    │    └─Conv2d: 3-1                       [1, 64, 17, 17]           9,472
│    │    └─BatchNorm2d: 3-2                  [1, 64, 17, 17]           128
│    │    └─ReLU: 3-3                         [1, 64, 17, 17]           --
│    │    └─MaxPool2d: 3-4                    [1, 64, 9, 9]             --
│    └─Sequential: 2-2                        [1, 192, 5, 5]            --
│    │    └─Conv2d: 3-5                       [1, 64, 9, 9]             4,160
│    │    └─BatchNorm2d: 3-6                  [1, 64, 9, 9]             128
│    │    └─ReLU: 3-7                         [1, 64, 9, 9]             --
│    │    └─Conv2d: 3-8                       [1, 192, 9, 9]            110,784
│    │ 

We then display the architecture of the student model. We can check that the number of convolution channels at each layer within all Inception modules is the same as the standard model. The model has $5.998.362$ trainable parameters. Thus, the number of parameters in the teacher model is about $31.5\%$ of the number of parameters in the student model.

In [8]:
# Display the architecture of the student network
torchinfo.summary(net2net.student_network, input_size=(1,
                                                       params.inceptionv2_cifar.NB_CHANNELS,
                                                       *params.inceptionv2_cifar.IMAGE_SHAPE))

Layer (type:depth-idx)                        Output Shape              Param #
GoogleNetBN                                   [1, 10]                   --
├─Sequential: 1-1                             [1, 10]                   --
│    └─Sequential: 2-1                        [1, 64, 9, 9]             --
│    │    └─Conv2d: 3-1                       [1, 64, 17, 17]           9,472
│    │    └─BatchNorm2d: 3-2                  [1, 64, 17, 17]           128
│    │    └─ReLU: 3-3                         [1, 64, 17, 17]           --
│    │    └─MaxPool2d: 3-4                    [1, 64, 9, 9]             --
│    └─Sequential: 2-2                        [1, 192, 5, 5]            --
│    │    └─Conv2d: 3-5                       [1, 64, 9, 9]             4,160
│    │    └─BatchNorm2d: 3-6                  [1, 64, 9, 9]             128
│    │    └─ReLU: 3-7                         [1, 64, 9, 9]             --
│    │    └─Conv2d: 3-8                       [1, 192, 9, 9]            110,784
│    │ 